<a href="https://colab.research.google.com/github/hanhanwu/Hanhan_COLAB_Experiemnts/blob/master/Try_DeepWalk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


I got generated the graph by using `seealsology`

* https://densitydesign.github.io/strumentalia-seealsology/
* It does web crawling for any wiki-page you typed in and creates a graph for you to download.
* I used "environment protection" and "global warming" 2 wiki pages

In [0]:
import networkx as nx
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
%matplotlib inline

In [12]:
graph_df = pd.read_csv("environment_wiki_graph.tsv", sep = "\t")
graph_df.head()

,source,target,depth
0,environmental protection,biodiversity,1
1,environmental protection,carbon offset,1
2,environmental protection,citizen science,1
3,environmental protection,conservation biology,1
4,environmental protection,conservation movement,1


In [13]:
graph_df['source'].value_counts()

environmentalism                        38
sustainable development                 38
sustainability                          34
natural resource management             34
ecology movement                        31
environmental movement                  31
natural capital                         27
environmental protection                25
conservation movement                   24
earth day                               23
environmental racism                    23
conservation biology                    20
anthropocene                            19
carbon offset                           19
citizen science                         13
public information and participation    13
participation (decision making)         13
green solutions                         13
planetary boundaries                    13
holocene extinction                     13
renewable resource                      11
biodiversity                             9
environmental governance                 9
global cool

In [14]:
# construct the graph
G = nx.from_pandas_edgelist(graph_df, "source", "target", 
                            edge_attr=True, create_using=nx.Graph())
G

In [15]:
# The number of nodes
len(G)

367

In [0]:
def random_walk(source_node, sequence_len):
  """
  For a souce node, get its neighbours that not in the random-walk sequence,
  randomly select a neighbour as the next source node,
  repeat until reaches to the max length or there is no node to choose.

  NOTE: No need to set seed for random, cuz we want each time to generate
        a different sequence when call this function.
  """
  random_walk_sequence = [source_node]

  for i in range(sequence_len - 1):
    target_nodes = list(G.neighbors(source_node))
    target_nodes = list(set(target_nodes) - set(random_walk_sequence))
    if len(target_nodes) == 0:
      break

    selected_node = random.choice(target_nodes)
    random_walk_sequence.append(selected_node)
    source_node = selected_node

  return random_walk_sequence

In [43]:
random_walk('sustainable development', 7)

['sustainable development',
 'circles of sustainability',
 'sustainability',
 'world cities summit']

In [40]:
# get random_walk sequences for all the nodes
all_nodes = list(G.nodes)

random_walks = []
for n in tqdm(all_nodes):  # tqdm is to show the progress
  for i in range(4):
    random_walks.append(random_walk(n, 10))

print(len(random_walks))

100%|██████████| 367/367 [00:00<00:00, 14716.01it/s]

1468


In [41]:
for seq in random_walks[4:10]:
  print(seq)

['biodiversity', 'deforestation and climate change']
['biodiversity', 'defaunation', 'anthropocene', 'meghalayan']
['biodiversity', 'bioversity international']
['biodiversity', "world scientists' warning to humanity", 'anthropocene', 'novel ecosystem']
['carbon offset', 'carbon sequestration']
['carbon offset', 'personal carbon trading']
